# 16. Linear Data Stuctures

## Lists
---

### The "List" Abstract Data Structure 

#### What is an **Abstract Data Type** (**ADT**)

In general, **abstract data types** (**ADT**) gives us a **definition** (**abstraction**) of the specific structure, i.e. **defines the allowed operations and properties**, without being interested in the specific implementation. This allows an **abstract data type** to have several different implementations and respectively different efficiency.

<br>

#### Introducing the `System.Collections.IList` Interface

Interfaces in C# construct a frame (contract) for their implementations – classes.    
   
This contract consists of **a set of methods and properties**, which
each class must implement in order to implement the interface.

Each **ADT** defines some interface. Let’s consider the interface `System.Collections.IList`.     
The **basic methods** which it defines are:

- `int Add(object)` – adds element in the end of the list 
- `void Insert(int, object)` – adds element on a chosen position in the list
- `void Clear()` – removes all elements in the list 
- `bool Contains(object)` – checks whether the list contains the element 
- `void Remove(object)` – removes the element from the list 
- `void RemoveAt(int)` – removes the element on a given position 
- `int IndexOf(object)` – returns the position of the element 
- `this[int]` – indexer, allows access to the elements on a set position

<br>

<br>

### Static Lists (An Array-Based Implementation)

##### Lists v.s. Arrays

Somewhat similarly to <a href="../07. Arrays/The Anatomy of an Array.ipynb">Arrays</a>, we could imagine the **list** as an **ordered sequence (line) of elements**.

A List features a length property (count of elements), and though it's elements are arranged consecutively, their addresses in memory are, unlike Arrays, non-contiguous (they don't live right next to each other).

Notwithsatnding, **Arrays perform many of the functions of the list ADT**, but there is an even more significant difference to consider – that Lists allow adding new elements, while **Arrays have fixed capacity**.

<br>

##### The Resizable Array-based List

Even so, there exists implementation techniques for an Array what make it possible to **automatically resize it's capacity** (in a similar fashion to the [StringBuilder](../13.%20Strings%20and%20Text%20Processing/Building%20Strings.ipynb) class).   

Such a list may be reffered to as a **Static List implemented with an extensible Array**.

Using [Generics](../14.%20Defining%20Classes/Generics.ipynb), we may define such an Array-based structure able to hold any type of data, as demonstrated below:

In [1]:
public class ResizableArray<T>
{

    // Field Definitions --------------------------------------------------
    private T[] array;
    private int occupiedElements;
    private const int DEFAULT_CAPACITY = 4;
    //---------------------------------------------------------------------


    // Property Definitions -----------------------------------------------
    public int OccupiedElements
    {
        // Read Only
        get
        {
            return this.occupiedElements;
        }
    }
    //---------------------------------------------------------------------


    // Constructor --------------------------------------------------------
    public ResizableArray( int capacity = DEFAULT_CAPACITY )
    {
        this.array = new T[ capacity ];
        this.occupiedElements = 0;
    }
    //---------------------------------------------------------------------


    // Methods ------------------------------------------------------------
    // Double the size of the array, should it be determined to be full
    public void ResizeIfArrayIsFull()
    {

        // IF the count of occupied elements, 
        // once adusted for zero indexing, 
        // and then subsequently incremented,
        // is found exceed the capacity of the array:
        if( this.OccupiedElements + 1 > this.array.Length )
        {

            // Declare a new array with twice the capacity
            T[] resizedArray = new T[ this.array.Length * 2 ];


            // Copy the occupied Array elements into the resized array
            Array.Copy(this.array, resizedArray, this.OccupiedElements); 


            // Point the Copy's object reference back to the orginal Array
            this.array = resizedArray;

        }

    }
    //---------------------------------------------------------------------
    // Append a new element to the end of the array
    public void Append( T elementToAppend )
    {

        // Double the size of the array, should it be determined to be full
        ResizeIfArrayIsFull();


        // Assign the given argument as the last element of the array,
        // taking care to simultaneously increment 
        // the count of occupied elements
        this.array[ this.occupiedElements++ ] = elementToAppend;

    }
    //---------------------------------------------------------------------
    // Insert a new element in the array at the specified index
    public void Insert( int indexToInsert, T elementToInsert )
    {

        // Try to insert the given element using the specified index,
        // proceeding cautiously in the event that the given index
        // could possibly be out of the boundaries contained by the Array
        try
        {

            // Double the size of the array, 
            // should it be determined to be full
            ResizeIfArrayIsFull();


            // Shift the each of the elements after the specified index
            // one position to the right in order to accomodate the space
            // needed to insert the specified element
            Array.Copy(
                this.array, indexToInsert,
                this.array, indexToInsert + 1,
                this.occupiedElements - indexToInsert
            );


            // Clobber the element currently at the specified index
            // with the specified element
            this.array[ indexToInsert ] = elementToInsert;


            // Increment the the count of occupied elements
            this.occupiedElements++; 

        }


        // Should there be an error, 
        // it's sure to be that the specified index is out of range
        // so provide an appropriate exception which generates a 
        // message to the user
        catch ( IndexOutOfRangeException ioor )
        {
            Console.WriteLine(
                $"Index: {indexToInsert} is invalid\n\n"
                +
                $"Stack Trace: {ioor.StackTrace}"
            );
        }

    }
    //---------------------------------------------------------------------



}

<table style="margin-left: auto; margin-right: auto;">
    <tbody>
        <tr>
            <td>1</td>
            <td>2</td>
            <td>3</td>
            <td style="background: blue; color: white;">4</td>
            <td style="background: red; color: white;">5</td>
            <td>6</td>
            <td>7</td>
        </tr>
        <tr>
            <td style="border: none;">0</td>
            <td style="border: none;">1</td>
            <td style="border: none;">2</td>
            <td style="font-weight: bold; border: none;">3</td>
            <td style="font-weight: bold; border: none">4</td>
            <td style="border: none">5</td>
            <td style="border: none">6</td>
        </tr>
    </tbody>
</table>

$$\Huge{\downarrow}$$

<table style="margin-left: auto; margin-right: auto;">
    <tbody>
        <tr>
            <td>1</td>
            <td>2</td>
            <td>3</td>
            <td style="background: blue; color: white;">4</td>
            <td style="background: red; color: white;">4</td>
            <td style="background: red; color: white;">5</td>
            <td style="background: red; color: white;">6</td>
            <td style="background: red; color: white;">7</td>
            <td>0</td>
            <td>0</td>
            <td>0</td>
            <td>0</td>
            <td>0</td>
            <td>0</td>
        </tr>
        <tr>
            <td style="border: none;">0</td>
            <td style="border: none;">1</td>
            <td style="border: none;">2</td>
            <td style="font-weight: bold; border: none;">3</td>
            <td style="font-weight: bold; border: none">4</td>
            <td style="font-weight: bold; border: none">5</td>
            <td style="font-weight: bold; border: none">6</td>
            <td style="font-weight: bold; border: none">7</td>
            <td style="border: none">8</td>
            <td style="border: none">9</td>
            <td style="border: none">10</td>
            <td style="border: none">11</td>
            <td style="border: none">12</td>
            <td style="border: none">13</td>
        </tr>
    </tbody>
</table>

$$\Huge{\downarrow}$$

<table style="margin-left: auto; margin-right: auto;">
    <tbody>
        <tr>
            <td>1</td>
            <td>2</td>
            <td>3</td>
            <td style="background: green; color: white;">&infin;</td>
            <td style="background: red; color: white;">4</td>
            <td style="background: red; color: white;">5</td>
            <td style="background: red; color: white;">6</td>
            <td style="background: red; color: white;">7</td>
            <td>0</td>
            <td>0</td>
            <td>0</td>
            <td>0</td>
            <td>0</td>
            <td>0</td>
        </tr>
        <tr>
            <td style="border: none;">0</td>
            <td style="border: none;">1</td>
            <td style="border: none;">2</td>
            <td style="font-weight: bold; border: none; color: green">3</td>
            <td style="font-weight: bold; border: none;">4</td>
            <td style="font-weight: bold; border: none">5</td>
            <td style="font-weight: bold; border: none">6</td>
            <td style="font-weight: bold; border: none">7</td>
            <td style="border: none">8</td>
            <td style="border: none">9</td>
            <td style="border: none">10</td>
            <td style="border: none">11</td>
            <td style="border: none">12</td>
            <td style="border: none">13</td>
        </tr>
    </tbody>
</table>